# Mini-projet – Débruitage de maillage 3D

IMI – Optimisation et problèmes inverses  
S8 – 2024/2025  

## Objectif

On cherche à débruiter un maillage triangulé en résolvant le problème :


xhat = argmin_(x in {R}^{N**3}) 
[||x - z||^2 + lambda R(x)]

où :
- \( z \) est le maillage bruité
- \( R(x) \) est une régularisation sur le graphe du maillage
- \( lambda \) est un paramètre de régularisation


# Imports

In [ ]:
# ===============================
# Bibliothèques Maths
# ===============================
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla

# ===============================
# Processing 3D
# ===============================
import igl  # libigl python binding

# ===============================
# Visualisation
# ===============================
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# ===============================
# IMICPE
# ===============================
import imicpe
import imicpe.optim as optim

print("imicpe version:", imicpe.__version__)


imicpe version: 1.0.15


# Lecture du maillage

In [ ]:
# Chemin vers le maillage
mesh_path = "teapot.off"

# Lecture
V, F = igl.read_triangle_mesh(mesh_path)

print("Nombre de sommets :", V.shape[0])
print("Nombre de faces :", F.shape[0])

Nombre de sommets : 1976
Nombre de faces : 3752


# Visualisation du maillage

In [ ]:
def plot_mesh(V, F, title="Mesh"):
    x, y, z = V[:, 0], V[:, 1], V[:, 2]
    i, j, k = F[:, 0], F[:, 1], F[:, 2]

    fig = go.Figure(data=[go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color='cyan',
        opacity=0.5
    )])

    fig.update_layout(title=title, scene=dict(aspectmode='data'))
    fig.show()

plot_mesh(V, F, "Maillage original")

# Ajout de bruit Gaussien

In [ ]:
VB = []
for i in range(len(V)) :
    n = np.random.normal(0, 0.5, len(V[i]))
    xbar = np.array(V[i])
    z = xbar + n
    VB.append(z)

plot_mesh(VB, F, "Maillage bruité")

-2.98711
-2.5017776074817633


TypeError: list indices must be integers or slices, not tuple

# Construction du graphe + Laplacien

# Modèle d'optimisation

## Modèle choisi



# Résolution

# Analyse de l'influence de lambda

## Analyse

- Petit λ → peu de régularisation → bruit encore visible
- Grand λ → surface très lissée → perte de détails
- Compromis à trouver

Sensibilité forte aux amplitudes en 3D (cf. énoncé).
